<center><h1>Thakur_Hiren_HW3</h1></center>
<br>
<br>

Name: Hiren Thakur
<br>
Github Username: hirenthakur
<br>
USC ID: 9304219569

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [6]:
import numpy as np
import os
import pandas as pd
from scipy.stats import bootstrap
import warnings
warnings.filterwarnings('ignore')

Get the AReM Data Set

Downloaded the dataset and uploaded in github data folder

### (b) Test and Train Data

In [10]:
def split_train_test_data(base_path):
  
    train_data = pd.DataFrame()
    test_data = pd.DataFrame()
       
    activity_folders = [folder for folder in os.listdir(base_path) if 'pdf' not in folder]

    for activity in activity_folders:
        activity_path = os.path.join(base_path, activity)
        activity_files = os.listdir(activity_path)

  
        if activity in ['bending1', 'bending2']:
            for file in activity_files:
                file_path = os.path.join(activity_path, file)
                data = pd.read_csv(file_path, skiprows=4, usecols=range(7))

                
                if file in ['dataset1.csv', 'dataset2.csv']:
                    test_data = pd.concat([test_data, data], ignore_index=True)
                else:
                    train_data = pd.concat([train_data, data], ignore_index=True)

        
        else:
            for file in activity_files:
                file_path = os.path.join(activity_path, file)
                data = pd.read_csv(file_path, skiprows=4, usecols=range(7))

            
                if file in ['dataset1.csv', 'dataset2.csv', 'dataset3.csv']:
                    test_data = pd.concat([test_data, data], ignore_index=True)
                else:
                    train_data = pd.concat([train_data, data], ignore_index=True)

    
    return train_data, test_data


base_path = "../data/AReM/"
train_df, test_df = split_train_test_data(base_path)


print("Training Data Sample:")
print(train_df.head())
print("\nTest Data Sample:")
print(test_df.head())


Training Data Sample:
   # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                0      42.00       0.71      21.25       0.43      30.00   
1              250      41.50       0.50      20.25       1.48      31.25   
2              500      41.50       0.50      14.25       1.92      33.00   
3              750      40.75       0.83      15.75       0.43      33.00   
4             1000      40.00       0.71      20.00       2.74      32.75   

   var_rss23  
0       0.00  
1       1.09  
2       0.00  
3       0.00  
4       0.43  

Test Data Sample:
   # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                0      39.25       0.43      22.75       0.43      33.75   
1              250      39.25       0.43      23.00       0.00      33.00   
2              500      39.25       0.43      23.25       0.43      33.00   
3              750      39.50       0.50      23.00       0.71      33.00   
4             1000   

### (c) Feature Extraction

#### i. Research

Mean: The average value of the time series, providing information on the overall trend or central tendency of the data.

Median: The middle value of the ordered data points, offering a measure of central tendency that is less affected by outliers compared to the mean.

Minimum: The smallest value in the time series, giving insight into the lower bound of the data's amplitude.

Maximum: The largest value in the time series, showing the upper bound of the data's amplitude.

Standard Deviation: Measures the amount of variation or dispersion from the mean, indicating how spread out the time series values are.

Variance: The square of the standard deviation, representing the spread of the data. High variance indicates a wide range of values.

Range: The difference between the maximum and minimum values, describing the amplitude range of the signal.

Skewness: A measure of the asymmetry of the data distribution. Positive skewness means the tail on the right side is longer, while negative skewness indicates a longer left tail.

Kurtosis: Describes the "tailedness" of the distribution. A high kurtosis means there are more extreme values, and low kurtosis indicates a more uniform distribution.

Root Mean Square (RMS): The square root of the mean of the squared values, useful for quantifying the magnitude of the time series signal.
Autocorrelation: Measures how related the time series is to a lagged version of itself. It helps detect periodic patterns or the regularity of the signal.

Energy: The sum of the squared values of the time series, representing the signal's total power.

Entropy: A measure of complexity or unpredictability in the time series, with higher entropy indicating more disorder.

Zero-Crossing Rate: The rate at which the time series crosses the zero-value line. It can be useful for identifying changes in the signal or oscillations.

Absolute Sum of Changes: The sum of the absolute differences between consecutive values, indicating the amount of variability or smoothness in the signal.

Interquartile Range (IQR): The difference between the 75th and 25th percentiles of the data, providing a measure of the spread of the central 50% of values.

Signal Magnitude Area (SMA): The sum of the absolute values of the signal, often used in activity recognition to measure overall signal strength.

Peak-to-Peak Amplitude: The difference between the maximum and minimum peak values in the time series, indicating the signal's oscillation range.

#### ii. Extraction

In [15]:
def fetch_time_series_data(folder_path):
    time_series_data = []

    def read_csv(file, path):
        return pd.read_csv(os.path.join(path, file), skiprows=4, usecols=range(1, 7))

    for directory in filter(lambda x: 'pdf' not in x, os.listdir(folder_path)):
        complete_path = os.path.join(folder_path, directory)
        csv_files = filter(lambda f: f.endswith('.csv'), os.listdir(complete_path))

        time_series_data.extend(map(lambda file: read_csv(file, complete_path), csv_files))

    return time_series_data

stats_features = ['min', 'max', 'mean', 'median', 'std_dev', '1st_quartile', '3rd_quartile']

data_columns = [f"{feature}{i}" for i in range(1, 7) for feature in stats_features]

DATA_PATH = "../data/AReM/"

loaded_time_series = fetch_time_series_data(DATA_PATH)

rows_of_features = []

for data in loaded_time_series:
    statistics = data.describe()
    row_data = []

    for i in range(6):
        row_data.extend([
            statistics.iloc[3, i],  
            statistics.iloc[7, i],  
            statistics.iloc[1, i],  
            statistics.iloc[5, i],  
            statistics.iloc[2, i],  
            statistics.iloc[4, i],  
            statistics.iloc[6, i]   
        ])
    
    rows_of_features.append(row_data)

df_features = pd.DataFrame(rows_of_features, columns=data_columns)

df_features.index = df_features.index + 1

df_features


,min1,max1,mean1,median1,std_dev1,1st_quartile1,3rd_quartile1,min2,max2,mean2,...,std_dev5,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std_dev6,1st_quartile6,3rd_quartile6
1,37.25,45.00,40.624792,40.500,1.476967,39.25,42.0000,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.0,1.92,0.570583,0.430,0.582915,0.00,1.3000
2,38.00,45.67,42.812812,42.500,1.435550,42.00,43.6700,0.0,1.22,0.372437,...,1.995255,32.0000,34.50,0.0,3.11,0.571083,0.430,0.601010,0.00,1.3000
3,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,1.999604,35.3625,36.50,0.0,1.79,0.493292,0.430,0.513506,0.00,0.9400
4,33.00,47.75,42.179812,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.0,2.18,0.613521,0.500,0.524317,0.00,1.0000
5,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.0,1.79,0.383292,0.430,0.389164,0.00,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,20.75,46.25,34.763333,35.290,4.742208,31.67,38.2500,0.0,12.68,4.223792,...,3.174681,14.2500,18.33,0.0,9.39,3.288271,3.270,1.647528,2.05,4.3050
85,21.50,51.00,34.935812,35.500,4.645944,32.00,38.0625,0.0,12.21,4.115750,...,3.192058,14.2375,18.25,0.0,10.21,3.280021,3.015,1.700918,2.12,4.5000
86,18.33,47.67,34.333042,34.750,4.948770,31.25,38.0000,0.0,12.48,4.396958,...,3.000493,13.7500,18.00,0.0,8.01,3.261583,2.980,1.617290,2.05,4.3200
87,18.33,45.75,34.599875,35.125,4.731790,31.50,38.0000,0.0,15.37,4.398833,...,2.905688,14.0000,18.25,0.0,8.86,3.289542,3.015,1.680170,2.12,4.2600


#### iii. Standard Deviation

In [17]:
def calculate_std(data):
    return np.std(data, ddof=1)

def bootstrap_ci(data, num_iterations=1000, ci=90):
    n = len(data)
    bootstrap_samples = np.random.choice(data, (num_iterations, n), replace=True)
    bootstrap_statistics = np.std(bootstrap_samples, axis=1, ddof=1)
    
    lower_bound = (100 - ci) / 2
    upper_bound = 100 - lower_bound
    ci_lower = np.percentile(bootstrap_statistics, lower_bound)
    ci_upper = np.percentile(bootstrap_statistics, upper_bound)
    
    return ci_lower, ci_upper

bootstrap_intervals = {}

for feature in df_features.columns:
    data = df_features[feature].values
    
    std_value = calculate_std(data)
    print(f"Standard deviation of {feature}: {std_value}")
    
    lower_ci, upper_ci = bootstrap_ci(data)
    bootstrap_intervals[feature] = (lower_ci, upper_ci)


Standard deviation of min1: 9.569975284512807
Standard deviation of max1: 4.39436168470319
Standard deviation of mean1: 5.3357176583441746
Standard deviation of median1: 5.440054137152637
Standard deviation of std_dev1: 1.772153397502841
Standard deviation of 1st_quartile1: 6.1535897241853705
Standard deviation of 3rd_quartile1: 5.138924613450999
Standard deviation of min2: 0.0
Standard deviation of max2: 5.0627293748106394
Standard deviation of mean2: 1.5741639200745141
Standard deviation of median2: 1.4122441649745636
Standard deviation of std_dev2: 0.8841054781862713
Standard deviation of 1st_quartile2: 0.9463861911256928
Standard deviation of 3rd_quartile2: 2.12526640640591
Standard deviation of min3: 2.956462059205819
Standard deviation of max3: 4.875136693954745
Standard deviation of mean3: 4.008380131777648
Standard deviation of median3: 4.036396318784075
Standard deviation of std_dev3: 0.9467102620752954
Standard deviation of 1st_quartile3: 4.22065788450763
Standard deviation o

In [18]:

for feature, ci in bootstrap_intervals.items():
    print(f"90% CI for the standard deviation of {feature}: {ci}")



90% CI for the standard deviation of min1: (8.26181150981084, 10.794370722608141)
90% CI for the standard deviation of max1: (3.387482764699266, 5.287967893457751)
90% CI for the standard deviation of mean1: (4.709953295616737, 5.8948303320218045)
90% CI for the standard deviation of median1: (4.782114701904006, 5.9483679430083445)
90% CI for the standard deviation of std_dev1: (1.5830345996593709, 1.9508023414834197)
90% CI for the standard deviation of 1st_quartile1: (5.583932701539521, 6.651039747083147)
90% CI for the standard deviation of 3rd_quartile1: (4.355384854379679, 5.8372320022459485)
90% CI for the standard deviation of min2: (0.0, 0.0)
90% CI for the standard deviation of max2: (4.637126039937011, 5.405852542496656)
90% CI for the standard deviation of mean2: (1.3861916366800298, 1.7004174060119293)
90% CI for the standard deviation of median2: (1.2318982106398386, 1.5522980057122957)
90% CI for the standard deviation of std_dev2: (0.8008751336466153, 0.9406821718754274)

#### iv. Select Features

Mean: It provides the central tendency in a time series relating to the "average" value. It gives us a hint about the overall intensity of the activity or magnitude of the signal.

Standard Deviation 

The standard deviation represents how values deviate from the mean. It captures variability or fluctuation in the time series, which critical in distinguishing activities because of steady signals versus those withave more variation, such as sitting versus walki
ng.
Maximum or  n:

The maxima/minima values of the time series capture the extremes. It gives an idea of the range and amplitude of the activity, hence distinguishing between high and low-intensity movements.

## 2. ISLR 3.7.4

### (a) Linear Train

The Residual Sum of Squares (RSS) for cubic regression is generally lower than that for linear regression. 

This is because cubic regression offers more flexibility and can better capture the complexity of the training data, leading to a more accurate fit and a reduced RSS compared to linear regression.

### (b) Linear Test

The RSS for linear regression is typically lower than that for cubic regression on the test data. 

Cubic regression tends to overfit the training data, leading to less accurate predictions on unseen test data than the more generalizable predictions from the linear regression model.

### (c) Not Linear Train

The RSS for cubic regression is generally smaller than that for linear regression.

This is because the cubic regression model has more flexibility, allowing it to fit the training data more closely, which results in a lower RSS compared to the linear regression model.

### (d) Not Linear Testing

The information provided is not enough to draw a conclusion.

For test data, the RSS will depend on the nature of the relationship in the data. If the data follows a linear pattern, linear regression will likely produce a lower RSS than cubic regression. However, if the data deviates significantly from linearity, cubic regression may result in a lower RSS compared to linear regression.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 

Refrences-

https://docs.python.org/3/library/os.html#os.listdir

https://docs.python.org/3/library/os.path.html#os.path.join

https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

https://pandas.pydata.org/docs/reference/api/pandas.concat.html

https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions

https://docs.python.org/3/library/functions.html#filter

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.std.html

https://numpy.org/doc/stable/reference/generated/numpy.std.html

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html

https://numpy.org/doc/stable/reference/generated/numpy.std.html
